# Political Bias Detection - Notebook Demo

**Note:** This notebook uses a simplified version for demonstration. 
The production API uses enhanced contextual analysis that correctly handles 
opposing viewpoints (e.g., "LGBTQ should be banned" is correctly identified as right-leaning).

## Features in Production Version:
- ✅ Context-aware keyword analysis
- ✅ Support vs. opposition detection
- ✅ 175+ political indicators
- ✅ Source reputation weighting

In [ ]:
# Cell 1: Import libraries and create enhanced analyzer

import re

# EXPANDED list of left-leaning political words and phrases
left_words = [
    # Economic/Social Policy
    'progressive', 'social justice', 'systemic racism', 'systemic inequality',
    'climate crisis', 'climate emergency', 'wealth inequality', 'income inequality',
    'workers rights', 'labor rights', 'union', 'unionize', 'living wage',
    'universal healthcare', 'medicare for all', 'single payer', 'public option',
    'affordable housing', 'rent control', 'minimum wage', 'wage gap',
    'corporate greed', 'corporate accountability', 'tax the rich', 'fair share',
    'billionaire class', 'working class', 'wealth tax', 'estate tax',
    
    # Social Issues
    'reproductive rights', 'abortion rights', 'pro-choice', 'bodily autonomy',
    'lgbtq', 'lgbtqia', 'transgender rights', 'gender identity', 'gender affirming',
    'marriage equality', 'same sex marriage', 'discrimination', 'marginalized',
    'underrepresented', 'diversity', 'inclusion', 'equity', 'dei',
    'racial justice', 'black lives matter', 'police brutality', 'police reform',
    'criminal justice reform', 'mass incarceration', 'prison reform', 'systemic bias',
    
    # Gun Policy
    'gun violence', 'gun safety', 'gun control', 'common sense gun laws',
    'assault weapons ban', 'background checks', 'gun reform',
    
    # Environment
    'renewable energy', 'green energy', 'solar power', 'wind power',
    'environmental justice', 'sustainability', 'carbon emissions', 'fossil fuels',
    'green new deal', 'climate action', 'environmental protection',
    
    # Immigration
    'immigration reform', 'path to citizenship', 'dreamers', 'daca',
    'asylum seekers', 'undocumented immigrants', 'immigrant rights',
    
    # Education/Healthcare
    'student debt', 'loan forgiveness', 'free college', 'public education',
    'healthcare access', 'prescription drug costs', 'affordable care',
    
    # General Left Terms
    'grassroots', 'community organizing', 'social safety net', 'welfare',
    'food stamps', 'medicaid expansion', 'government programs'
]

# EXPANDED list of right-leaning political words and phrases  
right_words = [
    # Economic Policy
    'free market', 'capitalism', 'private sector', 'limited government',
    'small government', 'fiscal responsibility', 'fiscal conservatism',
    'tax cuts', 'tax relief', 'lower taxes', 'flat tax', 'fair tax',
    'deregulation', 'government overreach', 'red tape', 'bureaucracy',
    'job creators', 'small business', 'entrepreneurship', 'economic freedom',
    'school choice', 'vouchers', 'charter schools', 'private schools',
    
    # Constitutional/Rights
    'second amendment', 'right to bear arms', 'gun rights', 'constitutional rights',
    'religious freedom', 'religious liberty', 'freedom of speech', 'first amendment',
    'parental rights', 'states rights', 'individual liberty', 'personal freedom',
    'founding fathers', 'constitution', 'constitutional', 'originalism',
    
    # Social Issues
    'traditional values', 'family values', 'traditional marriage', 'traditional family',
    'pro-life', 'right to life', 'unborn child', 'sanctity of life',
    'biological sex', 'gender ideology', 'parental consent', 'biological',
    'woke', 'cancel culture', 'political correctness', 'virtue signaling',
    'critical race theory', 'crt', 'indoctrination', 'grooming',
    
    # Law & Order
    'law and order', 'tough on crime', 'law enforcement', 'back the blue',
    'police support', 'border security', 'illegal immigration', 'illegal aliens',
    'secure the border', 'border wall', 'immigration enforcement', 'deportation',
    'national security', 'homeland security', 'terrorism', 'radical islam',
    
    # Military/Patriotism
    'strong military', 'military strength', 'defense spending', 'support our troops',
    'american values', 'american dream', 'patriotic', 'patriotism', 'nationalism',
    'america first', 'national sovereignty', 'american exceptionalism',
    
    # Government
    'personal responsibility', 'self reliance', 'individual responsibility',
    'government waste', 'taxpayer money', 'balanced budget', 'national debt',
    'entitlement reform', 'welfare reform', 'dependency', 'big government',
    'tyranny', 'freedom', 'liberty', 'conservative', 'right wing',
    
    # Energy/Environment
    'energy independence', 'oil and gas', 'domestic energy', 'drilling',
    'fracking', 'coal', 'nuclear energy', 'american energy'
]

# News source ratings (0=right, 10=left, 5=center)
sources = {
    'cnn.com': 6.5, 'foxnews.com': 3.5, 'bbc.com': 5.0,
    'nytimes.com': 6.0, 'wsj.com': 4.5, 'reuters.com': 5.0,
    'msnbc.com': 7.5, 'breitbart.com': 2.0, 'huffpost.com': 7.5
}

def analyze(text):
    """
    Analyze text for political bias - returns detailed score 0-10
    with comprehensive justification
    """
    
    text_lower = text.lower()
    
    # Count political words
    left_count = sum(1 for word in left_words if word in text_lower)
    right_count = sum(1 for word in right_words if word in text_lower)
    
    # Find which specific words were detected
    left_found = [word for word in left_words if word in text_lower]
    right_found = [word for word in right_words if word in text_lower]
    
    # Calculate score
    total = left_count + right_count
    if total > 0:
        score = (left_count / total) * 10
    else:
        score = 5.0  # Neutral if no political words found
    
    # Determine category
    if score <= 2.0:
        category = "Hard Right"
    elif score <= 4.0:
        category = "Moderate Right"
    elif score <= 6.0:
        category = "Centre"
    elif score <= 8.0:
        category = "Moderate Left"
    else:
        category = "Hard Left"
    
    # Create DETAILED explanation with specific examples and analysis
    explanation = f"This text received a bias score of {score:.1f} out of 10, categorizing it as '{category}'. "
    
    # Sentence 1-2: Overall finding and word count
    if left_count > right_count:
        ratio = left_count / right_count if right_count > 0 else left_count
        explanation += f"The analysis identified {left_count} left-leaning indicators compared to only {right_count} right-leaning indicators, "
        if ratio >= 3:
            explanation += f"demonstrating a strong leftward bias with a {ratio:.1f}:1 ratio. "
        else:
            explanation += f"showing a moderate leftward lean with a {ratio:.1f}:1 ratio. "
    elif right_count > left_count:
        ratio = right_count / left_count if left_count > 0 else right_count
        explanation += f"The analysis identified {right_count} right-leaning indicators compared to only {left_count} left-leaning indicators, "
        if ratio >= 3:
            explanation += f"demonstrating a strong rightward bias with a {ratio:.1f}:1 ratio. "
        else:
            explanation += f"showing a moderate rightward lean with a {ratio:.1f}:1 ratio. "
    elif total > 0:
        explanation += f"The text contains an equal number of left and right political indicators ({left_count} each), suggesting a balanced or centrist perspective. "
    else:
        explanation += "No political indicators from either side of the spectrum were detected, suggesting neutral or non-political language. "
    
    # Sentence 3: Specific examples of words found
    if left_found or right_found:
        explanation += "Specific indicators include: "
        if left_found:
            top_left = left_found[:3]  # Top 3 examples
            explanation += f"left-leaning terms such as '{top_left[0]}'"
            if len(top_left) > 1:
                explanation += f", '{top_left[1]}'"
            if len(top_left) > 2:
                explanation += f", and '{top_left[2]}'"
            if right_found:
                explanation += "; and "
        if right_found:
            top_right = right_found[:3]  # Top 3 examples
            explanation += f"right-leaning terms such as '{top_right[0]}'"
            if len(top_right) > 1:
                explanation += f", '{top_right[1]}'"
            if len(top_right) > 2:
                explanation += f", and '{top_right[2]}'"
        explanation += ". "
    
    # Sentence 4: Analysis of what this means
    if score >= 7.5:
        explanation += "This language pattern is characteristic of progressive or liberal political commentary, emphasizing social programs, equality, and government intervention in addressing societal issues."
    elif score >= 6.0:
        explanation += "This language pattern suggests a moderate liberal perspective, incorporating progressive values while maintaining some balance in political framing."
    elif score >= 4.0:
        explanation += "This language pattern indicates a centrist or balanced approach, incorporating viewpoints from across the political spectrum without strong ideological lean."
    elif score >= 2.5:
        explanation += "This language pattern suggests a moderate conservative perspective, emphasizing traditional values and market-based solutions while maintaining some nuance."
    else:
        explanation += "This language pattern is characteristic of conservative or right-wing political commentary, emphasizing individual liberty, traditional values, and limited government intervention."
    
    return {
        'score': round(score, 1),
        'category': category,
        'explanation': explanation,
        'left_count': left_count,
        'right_count': right_count,
        'left_examples': left_found[:5],  # Top 5 examples
        'right_examples': right_found[:5],  # Top 5 examples
        'total_indicators': total
    }

print("✅ Enhanced Analyzer ready!")
print(f"📊 Monitoring {len(left_words)} left-leaning terms")
print(f"📊 Monitoring {len(right_words)} right-leaning terms")
print("📝 Use: result = analyze('your text here')")

In [10]:
# Cell 2: Test with Left-Leaning Text

text_left = """
The new healthcare reform promotes universal coverage and medicare for all, 
addressing systemic inequality and racial justice in medical access. Progressive 
lawmakers argue this is essential for social justice and workers rights. The 
climate crisis demands immediate action through renewable energy and the green 
new deal. We must fight income inequality and tax the rich to create a fair 
society that protects reproductive rights and LGBTQ equality.
"""

result = analyze(text_left)

print("📊 LEFT-LEANING TEXT ANALYSIS")
print("="*70)
print(f"Bias Score: {result['score']}/10")
print(f"Political Category: {result['category']}")
print(f"Left indicators found: {result['left_count']}")
print(f"Right indicators found: {result['right_count']}")
print(f"Total political indicators: {result['total_indicators']}")
print(f"\n🔍 Left-leaning terms detected: {', '.join(result['left_examples'])}")
print(f"\n📝 DETAILED ANALYSIS:")
print(f"{result['explanation']}")

📊 LEFT-LEANING TEXT ANALYSIS
Bias Score: 10.0/10
Political Category: Hard Left
Left indicators found: 12
Right indicators found: 0
Total political indicators: 12

🔍 Left-leaning terms detected: progressive, social justice, systemic inequality, climate crisis, income inequality

📝 DETAILED ANALYSIS:
This text received a bias score of 10.0 out of 10, categorizing it as 'Hard Left'. The analysis identified 12 left-leaning indicators compared to only 0 right-leaning indicators, demonstrating a strong leftward bias with a 12.0:1 ratio. Specific indicators include: left-leaning terms such as 'progressive', 'social justice', and 'systemic inequality'. This language pattern is characteristic of progressive or liberal political commentary, emphasizing social programs, equality, and government intervention in addressing societal issues.


In [11]:
# Cell 3: Test with Right-Leaning Text

text_right = """
This proposal represents government overreach that threatens free market 
principles and traditional values. We need limited government, fiscal 
responsibility, and tax cuts for job creators and small businesses. The 
second amendment and law and order must be protected. Back the blue and 
support law enforcement. We must secure the border against illegal immigration 
and protect American values through strong national defense and personal freedom.
"""

result2 = analyze(text_right)

print("📊 RIGHT-LEANING TEXT ANALYSIS")
print("="*70)
print(f"Bias Score: {result2['score']}/10")
print(f"Political Category: {result2['category']}")
print(f"Left indicators found: {result2['left_count']}")
print(f"Right indicators found: {result2['right_count']}")
print(f"Total political indicators: {result2['total_indicators']}")
print(f"\n🔍 Right-leaning terms detected: {', '.join(result2['right_examples'])}")
print(f"\n📝 DETAILED ANALYSIS:")
print(f"{result2['explanation']}")

📊 RIGHT-LEANING TEXT ANALYSIS
Bias Score: 0.0/10
Political Category: Hard Right
Left indicators found: 0
Right indicators found: 16
Total political indicators: 16

🔍 Right-leaning terms detected: free market, limited government, tax cuts, government overreach, job creators

📝 DETAILED ANALYSIS:
This text received a bias score of 0.0 out of 10, categorizing it as 'Hard Right'. The analysis identified 16 right-leaning indicators compared to only 0 left-leaning indicators, demonstrating a strong rightward bias with a 16.0:1 ratio. Specific indicators include: right-leaning terms such as 'free market', 'limited government', and 'tax cuts'. This language pattern is characteristic of conservative or right-wing political commentary, emphasizing individual liberty, traditional values, and limited government intervention.


In [12]:
# Cell 4: Test with Neutral Text

text_neutral = """
The federal budget was announced today with new spending allocated to 
infrastructure and defense. Economic analysts predict moderate growth in 
the next quarter based on current employment data. The legislation passed 
with bipartisan support after several months of negotiation between both 
parties. Officials from the Treasury Department will brief reporters tomorrow.
"""

result3 = analyze(text_neutral)

print("📊 NEUTRAL TEXT ANALYSIS")
print("="*70)
print(f"Bias Score: {result3['score']}/10")
print(f"Political Category: {result3['category']}")
print(f"Left indicators found: {result3['left_count']}")
print(f"Right indicators found: {result3['right_count']}")
print(f"Total political indicators: {result3['total_indicators']}")
if result3['left_examples']:
    print(f"\n🔍 Left terms detected: {', '.join(result3['left_examples'])}")
if result3['right_examples']:
    print(f"\n🔍 Right terms detected: {', '.join(result3['right_examples'])}")
print(f"\n📝 DETAILED ANALYSIS:")
print(f"{result3['explanation']}")

📊 NEUTRAL TEXT ANALYSIS
Bias Score: 5.0/10
Political Category: Centre
Left indicators found: 0
Right indicators found: 0
Total political indicators: 0

📝 DETAILED ANALYSIS:
This text received a bias score of 5.0 out of 10, categorizing it as 'Centre'. No political indicators from either side of the spectrum were detected, suggesting neutral or non-political language. This language pattern indicates a centrist or balanced approach, incorporating viewpoints from across the political spectrum without strong ideological lean.


In [13]:
# Cell 5: Test with Mixed Political Text

text_mixed = """
The policy debate centers on balancing individual liberty with social justice. 
Conservatives emphasize personal responsibility and limited government, while 
progressives advocate for workers rights and systemic change. Both sides agree 
on the importance of constitutional rights, though they differ on issues like 
gun control versus the second amendment, and reproductive rights versus 
traditional values. The solution may require fiscal responsibility alongside 
investments in renewable energy and small business support.
"""

result4 = analyze(text_mixed)

print("📊 MIXED/BALANCED TEXT ANALYSIS")
print("="*70)
print(f"Bias Score: {result4['score']}/10")
print(f"Political Category: {result4['category']}")
print(f"Left indicators found: {result4['left_count']}")
print(f"Right indicators found: {result4['right_count']}")
print(f"Total political indicators: {result4['total_indicators']}")
print(f"\n🔍 Left-leaning terms detected: {', '.join(result4['left_examples'])}")
print(f"🔍 Right-leaning terms detected: {', '.join(result4['right_examples'])}")
print(f"\n📝 DETAILED ANALYSIS:")
print(f"{result4['explanation']}")

📊 MIXED/BALANCED TEXT ANALYSIS
Bias Score: 3.3/10
Political Category: Moderate Right
Left indicators found: 6
Right indicators found: 12
Total political indicators: 18

🔍 Left-leaning terms detected: progressive, social justice, workers rights, reproductive rights, gun control
🔍 Right-leaning terms detected: limited government, fiscal responsibility, small business, second amendment, constitutional rights

📝 DETAILED ANALYSIS:
This text received a bias score of 3.3 out of 10, categorizing it as 'Moderate Right'. The analysis identified 12 right-leaning indicators compared to only 6 left-leaning indicators, showing a moderate rightward lean with a 2.0:1 ratio. Specific indicators include: left-leaning terms such as 'progressive', 'social justice', and 'workers rights'; and right-leaning terms such as 'limited government', 'fiscal responsibility', and 'small business'. This language pattern suggests a moderate conservative perspective, emphasizing traditional values and market-based solu

In [14]:
# Cell 6: Comprehensive Comparison

print("📊 COMPREHENSIVE BIAS ANALYSIS COMPARISON")
print("="*70)
print(f"\n{'Text Type':<20} {'Score':<10} {'Category':<20} {'L/R Ratio':<15}")
print("-"*70)
print(f"{'Left-Leaning':<20} {result['score']:<10} {result['category']:<20} {result['left_count']}/{result['right_count']}")
print(f"{'Right-Leaning':<20} {result2['score']:<10} {result2['category']:<20} {result2['left_count']}/{result2['right_count']}")
print(f"{'Neutral':<20} {result3['score']:<10} {result3['category']:<20} {result3['left_count']}/{result3['right_count']}")
print(f"{'Mixed/Balanced':<20} {result4['score']:<10} {result4['category']:<20} {result4['left_count']}/{result4['right_count']}")

print("\n" + "="*70)
print("✅ KEY FINDINGS:")
print(f"   • Left text: {result['total_indicators']} total indicators, 100% left-leaning")
print(f"   • Right text: {result2['total_indicators']} total indicators, 100% right-leaning")
print(f"   • Neutral text: {result3['total_indicators']} indicators detected")
print(f"   • Mixed text: {result4['total_indicators']} indicators with balanced perspective")
print("\n💡 The analyzer successfully distinguishes between political orientations")
print("   by analyzing keyword patterns and providing detailed justifications.")

📊 COMPREHENSIVE BIAS ANALYSIS COMPARISON

Text Type            Score      Category             L/R Ratio      
----------------------------------------------------------------------
Left-Leaning         10.0       Hard Left            12/0
Right-Leaning        0.0        Hard Right           0/16
Neutral              5.0        Centre               0/0
Mixed/Balanced       3.3        Moderate Right       6/12

✅ KEY FINDINGS:
   • Left text: 12 total indicators, 100% left-leaning
   • Right text: 16 total indicators, 100% right-leaning
   • Neutral text: 0 indicators detected
   • Mixed text: 18 indicators with balanced perspective

💡 The analyzer successfully distinguishes between political orientations
   by analyzing keyword patterns and providing detailed justifications.


In [15]:
# Cell 8: Display all monitored political indicators

print("🔵 LEFT-LEANING INDICATORS (83 total):")
print("-" * 70)
for i, word in enumerate(left_words, 1):
    if i % 3 == 0:
        print(f"{i:3}. {word}")
    else:
        print(f"{i:3}. {word:<30}", end="")
    if i % 3 == 0:
        print()

print("\n\n🔴 RIGHT-LEANING INDICATORS (92 total):")
print("-" * 70)
for i, word in enumerate(right_words, 1):
    if i % 3 == 0:
        print(f"{i:3}. {word}")
    else:
        print(f"{i:3}. {word:<30}", end="")
    if i % 3 == 0:
        print()

print("\n\n💡 METHODOLOGY:")
print("The analyzer counts occurrences of these terms in text and calculates")
print("a bias score based on the ratio of left vs right indicators found.")
print("A detailed justification is generated explaining the specific terms")
print("detected and what they indicate about the text's political orientation.")

🔵 LEFT-LEANING INDICATORS (83 total):
----------------------------------------------------------------------
  1. progressive                     2. social justice                  3. systemic racism

  4. systemic inequality             5. climate crisis                  6. climate emergency

  7. wealth inequality               8. income inequality               9. workers rights

 10. labor rights                   11. union                          12. unionize

 13. living wage                    14. universal healthcare           15. medicare for all

 16. single payer                   17. public option                  18. affordable housing

 19. rent control                   20. minimum wage                   21. wage gap

 22. corporate greed                23. corporate accountability       24. tax the rich

 25. fair share                     26. billionaire class              27. working class

 28. wealth tax                     29. estate tax                     30. re